<a href="https://colab.research.google.com/github/034adarsh/Stock-Price-Prediction-Using-LSTM/blob/main/LSTM_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import all the required libraries

---



In [ ]:
# import libraries
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json
import os
import numpy as np
import tensorflow as tf # This code has been tested with TensorFlow 1.6
from sklearn.preprocessing import MinMaxScaler
import yfinance as yf

In [ ]:
# define the stock ticker and date ranges
ticker='QUBT'
start_date='2010-01-01'
end_date='2025-07-31'
training_data_start_date='2010-01-01'
training_data_end_date='2022-12-31'
validation_data_start_date='2023-01-01'
validation_data_end_date='2023-12-31'
test_data_start_date='2024-01-01'
test_data_end_date='2024-12-31'
prediction_start_date='2025-01-01'
# Download historical data for a stock from Yahoo Finance
df = yf.download(ticker, start=start_date, end=end_date)

In [ ]:
# Sort DataFrame by date
df = df.sort_values('Date')

# Double check the result
df.head()
df.tail()

In [ ]:
plt.figure(figsize = (18,9))
plt.plot(range(df.shape[0]),(df['Low']+df['High'])/2.0)
plt.xticks(range(0,df.shape[0],500),df['Date'].loc[::500],rotation=45)
plt.xlabel('Date',fontsize=18)
plt.ylabel('Mid Price',fontsize=18)
plt.show()


In [ ]:
# First calculate the mid prices from the highest and lowest
high_prices = df.loc[:,'High'].as_matrix()
low_prices = df.loc[:,'Low'].as_matrix()
mid_prices = (high_prices+low_prices)/2.0


In [ ]:
# train and test split
train_data = mid_prices[:11000]
test_data = mid_prices[11000:]


In [ ]:
# Scale the data to be between 0 and 1
# When scaling remember! You normalize both test and train data with respect to training data
# Because you are not supposed to have access to test data
scaler = MinMaxScaler()
train_data = train_data.reshape(-1,1)
test_data = test_data.reshape(-1,1)
smoothing_window_size = 2500
# We smooth the data and also scale it by gpt-4
for di in range(0,10000,smoothing_window_size):
    scaler.fit(train_data[di:di+smoothing_window_size,:])
    train_data[di:di+smoothing_window_size,:] = scaler.transform(train_data[di:di+smoothing_window_size,:])
scaler.fit(train_data[di+smoothing_window_size:,:])
train_data[di+smoothing_window_size:,:] = scaler.transform(train_data[di+smoothing_window_size:,:])
test_data = scaler.transform(test_data)